<a href="https://colab.research.google.com/github/racoope70/daytrading-with-ml/blob/main/multi_stock_sac_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Fix Potential Library Conflicts
!apt-get remove --purge -y cuda* libcuda* nvidia* || echo "No conflicting CUDA packages"
!apt-get autoremove -y
!apt-get clean

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Note, selecting 'cuda-toolkit-12-4-config-common' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-11-0' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-11-1' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-11-7' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-11-8' for glob 'cuda*'
Note, selecting 'cuda-toolkit-12-5-config-common' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-12-0' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-12-1' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-12-2' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-12-3' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-12-4' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-12-5' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-12-6' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-12-8' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-12-9' for glob 'cuda*'
Note, selecting 'cuda-toolkit-12-6-config

In [2]:
#Protocol Buffer Fix (for TensorFlow)
!pip uninstall -y protobuf
!pip install protobuf==3.20.3

Found existing installation: protobuf 5.29.5
Uninstalling protobuf-5.29.5:
  Successfully uninstalled protobuf-5.29.5
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 9.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.71.0 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 3.20.3 which is incompatible.
ydf 0.12.0 requires protobuf<6.0.0,>=5.29.1, but you have protobuf 3.20.3 which is incompatible.
tensorflow-metadata 1.17.1 requires protobuf<6.0.0,>=4.25.2; python_version >= "3.11", but you have protobuf 3.20.3 which is incompatible.


In [3]:
#Update Colab Environment and System Libraries
!apt-get update -y && apt-get upgrade -y


Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,553 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [3,295 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-security/ma

In [4]:
#Install Correct Version of CUDA for Colab GPU
!apt-get update -qq && apt-get install -y \
    libcusolver11 libcusparse11 libcurand10 libcufft10 libnppig10 libnppc10 libnppial10 \
    cuda-toolkit-12-4

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
E: Unable to locate package libnppig10
E: Unable to locate package libnppc10
E: Unable to locate package libnppial10


In [5]:
#Set Correct CUDA Paths
import os
os.environ['CUDA_HOME'] = '/usr/local/cuda-12.4'
os.environ['PATH'] += ':/usr/local/cuda-12.4/bin'
os.environ['LD_LIBRARY_PATH'] += ':/usr/local/cuda-12.4/lib64'


In [6]:
#Install RAPIDS and NVIDIA Dependencies
!pip install --extra-index-url=https://pypi.nvidia.com \
    cuml-cu12==25.2.0 cudf-cu12==25.2.0 cupy-cuda12x dask-cuda==25.2.0 dask-cudf-cu12==25.2.0


Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 31.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 39.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 48.4 MB/s eta 0:00:00
  Attempting uninstall: cudf-cu12
    Found existing installation: cudf-cu12 25.2.1
    Uninstalling cudf-cu12-25.2.1:
      Successfully uninstalled cudf-cu12-25.2.1
  Attempting uninstall: dask-cudf-cu12
    Found existing installation: dask-cudf-cu12 25.2.2
    Uninstalling dask-cudf-cu12-25.2.2:
      Successfully uninstalled dask-cudf-cu12-25.2.2
  Attempting uninstall: cuml-cu12
    Found existing installation: cuml-cu12 25.2.1
    Uninstalling cuml-cu12-25.2.1:
      Successfully uninstalled cuml-cu12-25.2.1


In [7]:
#Install TensorFlow (latest GPU-compatible version)
!pip install tensorflow==2.18.0

#Install Stable Baselines3 and Trading Libraries
!pip install stable-baselines3[extra] gymnasium gym-anytrading yfinance xgboost joblib

#Install Miscellaneous Libraries
!pip install matplotlib scikit-learn pandas numba==0.61.0

#Install PyTorch with GPU Support
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.2/172.2 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 59.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 45.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 34.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 94.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.5/184.5 kB 16.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-c

In [8]:
#Install TensorFlow (latest GPU-compatible version)
!pip install tensorflow==2.18.0

#Immediately follow it with:
import tensorflow as tf

gpus = tf.config.list_physical_devices("GPU")
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("TensorFlow GPU memory growth enabled")
    except RuntimeError as e:
        print(f"TensorFlow GPU memory config failed: {e}")


In [ ]:
!fusermount -u /content/drive

fusermount: failed to unmount /content/drive: No such file or directory


In [11]:
from google.colab import drive
drive.flush_and_unmount()  #  safely unmounts


Drive not mounted, so nothing to flush and unmount.


In [9]:
# === Imports ===
import os
import gc
import time
import json
import random
from datetime import datetime
from collections import deque

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import yfinance as yf

from stable_baselines3 import SAC, PPO
from stable_baselines3.common.noise import NormalActionNoise
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnNoModelImprovement



#Ticker List and CONFIG
ticker_list = [
    'AAPL', 'TSLA', 'MSFT', 'GOOGL', 'AMZN', 'NVDA', 'META', 'BRK-B', 'JPM', 'JNJ',
    'XOM', 'V', 'PG', 'UNH', 'MA', 'HD', 'LLY', 'MRK', 'PEP', 'KO',
    'BAC', 'ABBV', 'AVGO', 'PFE', 'COST', 'CSCO', 'TMO', 'ABT', 'ACN', 'WMT',
    'MCD', 'ADBE', 'DHR', 'CRM', 'NKE', 'INTC', 'QCOM', 'NEE', 'AMD', 'TXN',
    'AMGN', 'UPS', 'LIN', 'PM', 'UNP', 'BMY', 'LOW', 'RTX', 'CVX', 'IBM',
    'GE', 'SBUX', 'ORCL'
]

strategy_name = "sac_ppo_td3_multi_stock_v1"

CONFIG = {
    'symbols': [],
    'period': '720d',
    'interval': '1h',
    'target': 'Target',
    'sharpe_threshold': 1.5,
    'return_threshold': 1.25,
    'strategy_name': strategy_name
}

def download_stock_data(ticker, period="720d", interval="1h", max_retries=5):
    for attempt in range(1, max_retries + 1):
        try:
            print(f"Attempt {attempt}: Downloading {ticker}...")
            df = yf.download(ticker, period=period, interval=interval)
            if not df.empty:
                df.reset_index(inplace=True)
                df['Symbol'] = ticker
                return df
            raise ValueError("Empty data")
        except Exception as e:
            print(f"Error: {e}. Retrying in {attempt * 5} sec...")
            time.sleep(attempt * 5)
    print(f"Failed to download {ticker}")
    return None

#Feature Engineering Function
def compute_enhanced_features(df):
    df = df.copy()
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = df.columns.get_level_values(0)
    df = df.loc[:, ~df.columns.duplicated()]

    df['SMA_20'] = df['Close'].rolling(20).mean()
    df['STD_20'] = df['Close'].rolling(20).std()
    df['Upper_Band'] = df['SMA_20'] + 2 * df['STD_20']
    df['Lower_Band'] = df['SMA_20'] - 2 * df['STD_20']
    df['Lowest_Low'] = df['Low'].rolling(14).min()
    df['Highest_High'] = df['High'].rolling(14).max()
    denom = (df['Highest_High'] - df['Lowest_Low']).replace(0, np.nan)
    df['Stoch'] = ((df['Close'] - df['Lowest_Low']) / denom) * 100
    df['ROC'] = df['Close'].pct_change(10)
    df['OBV'] = (np.sign(df['Close'].diff()) * df['Volume']).cumsum()
    typical_price = (df['High'] + df['Low'] + df['Close']) / 3
    df['CCI'] = (typical_price - typical_price.rolling(20).mean()) / (0.015 * typical_price.rolling(20).std())
    df['PROC'] = ((df['Close'] - df['Close'].shift(12)) / df['Close'].shift(12)) * 100
    df['Rolling_Mean_50'] = df['Close'].rolling(50).mean()
    df['Expanding_Mean'] = df['Close'].expanding().mean()
    df['EMA_10'] = df['Close'].ewm(span=10).mean()
    df['EMA_50'] = df['Close'].ewm(span=50).mean()
    df['MACD_Line'] = df['Close'].ewm(span=12).mean() - df['Close'].ewm(span=26).mean()
    df['MACD_Signal'] = df['MACD_Line'].ewm(span=9).mean()
    df['MACD_Hist'] = df['MACD_Line'] - df['MACD_Signal']
    delta = df['Close'].diff()
    gain = delta.where(delta > 0, 0).rolling(14).mean()
    loss = -delta.where(delta < 0, 0).rolling(14).mean()
    rs = gain / loss
    df['RSI'] = 100 - (100 / (1 + rs))
    df['True_Range'] = df[['High', 'Low', 'Close']].apply(
        lambda x: max(x.iloc[0] - x.iloc[1], abs(x.iloc[0] - x.iloc[2]), abs(x.iloc[1] - x.iloc[2])), axis=1)
    df['ATR'] = df['True_Range'].rolling(14).mean()
    df['+DM'] = np.where((df['High'].diff() > df['Low'].diff()) & (df['High'].diff() > 0), df['High'].diff(), 0)
    df['-DM'] = np.where((df['Low'].diff() > df['High'].diff()) & (df['Low'].diff() > 0), df['Low'].diff(), 0)
    df['+DI'] = 100 * df['+DM'].rolling(14).mean() / df['ATR']
    df['-DI'] = 100 * df['-DM'].rolling(14).mean() / df['ATR']
    df['ADX'] = abs(df['+DI'] - df['-DI']).rolling(14).mean()
    df['Volume_Avg'] = df['Volume'].rolling(20).mean()
    df['Volume_Change'] = df['Volume'].pct_change()
    df['Volume_Change_MA'] = df['Volume_Change'].rolling(10).mean()
    df['Volume_Change_Ratio'] = df['Volume_Change'] / df['Volume_Change'].shift(1)
    df['Relative_Volume'] = df['Volume'] / df['Volume_Avg']
    df['Trailing_Stop'] = np.minimum(df['Close'] * 0.985, df['Close'] - (df['ATR'] * 0.3))
    df['Buy_Signal'] = np.where((df['RSI'] < 60) & (df['EMA_10'] > df['EMA_50']) &
                                ((df['MACD_Line'] > df['MACD_Signal']) | (df['MACD_Line'].diff() > 0)) &
                                (df['Volume'] > (0.4 * df['Volume_Avg'])) & (df['ADX'] > 18), 1, 0)
    df['Sell_Signal'] = np.where(((df['EMA_10'] < df['EMA_50']) & (df['RSI'] > 60)) |
                                 ((df['MACD_Line'] < df['MACD_Signal']) & (df['RSI'] > 65)) |
                                 (df['Close'] < df['Trailing_Stop']) |
                                 ((df['Volume'] > 0.5 * df['Volume_Avg']) & (df['ADX'] > 20)), 1, 0)
    df['Sell_Signal_Debug'] = np.where(((df['MACD_Hist'] < 0.5) | (df['MACD_Line'] < df['MACD_Signal'])) &
                                       (df['RSI'] < 55) & (df['ADX'] > 15) &
                                       ((df['Close'] < df['Trailing_Stop']) | (df['EMA_10'] < df['EMA_50'])) &
                                       (df['Volume'] > 0.5 * df['Volume_Avg']), 1, 0)
    df['Future_Close'] = df['Close'].shift(-10)
    df['Volatility'] = df['Close'].pct_change().rolling(window=20).std()
    df['Return'] = (df['Future_Close'] - df['Close']) / df['Close']
    df['Target'] = np.select([df['Return'] > 0.02, df['Return'] < -0.02], [1, -1], default=0)
    df['Multi_Class_Target'] = df['Target']
    df['Hour'] = pd.to_datetime(df['Datetime']).dt.hour
    df['DayOfWeek'] = pd.to_datetime(df['Datetime']).dt.dayofweek
    df['Session'] = np.where((df['Hour'] >= 9) & (df['Hour'] <= 16), 'Regular',
                             np.where((df['Hour'] < 9), 'Pre-market', 'After-hours'))
    df['MACD_Crossover'] = np.where(df['MACD_Line'] > df['MACD_Signal'], 1, 0)
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    df.dropna(inplace=True)
    return df

all_dfs = []

for ticker in ticker_list:
    df_single = download_stock_data(ticker, period=CONFIG['period'], interval=CONFIG['interval'])
    if df_single is not None:
        try:
            df_features = compute_enhanced_features(df_single)
            all_dfs.append(df_features)
        except Exception as e:
            print(f"Feature engineering failed for {ticker}: {e}")
    else:
        print(f"Failed to download {ticker}")

if all_dfs:
    df = pd.concat(all_dfs, ignore_index=True)
    print(f"Combined dataset created with shape: {df.shape}")
else:
    df = pd.DataFrame()
    print("No data available.")

if not df.empty:
    df.to_csv("multi_stock_feature_engineered_dataset.csv", index=False)
    print("Saved locally to multi_stock_feature_engineered_dataset.csv")

    drive_path = "/content/drive/MyDrive/trading_data/"
    os.makedirs(drive_path, exist_ok=True)
    df.to_csv(os.path.join(drive_path, "multi_stock_feature_engineered_dataset.csv"), index=False)
    print(f"Also saved to Google Drive at {drive_path}multi_stock_feature_engineered_dataset.csv")



Attempt 1: Downloading AAPL...
YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


Attempt 1: Downloading TSLA...


[*********************100%***********************]  1 of 1 completed


Attempt 1: Downloading MSFT...


...

Attempt 1: Downloading SBUX...


[*********************100%***********************]  1 of 1 completed


Attempt 1: Downloading ORCL...


[*********************100%***********************]  1 of 1 completed


Combined dataset created with shape: (262056, 51)
Saved locally to multi_stock_feature_engineered_dataset.csv
Also saved to Google Drive at /content/drive/MyDrive/trading_data/multi_stock_feature_engineered_dataset.csv


In [ ]:
# === Imports ===
import os, gc, json, time, joblib, random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import yfinance as yf
from datetime import datetime, timedelta
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from google.colab import drive
from stable_baselines3 import SAC
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnNoModelImprovement
from stable_baselines3.common.noise import NormalActionNoise
from gym_anytrading.envs import StocksEnv
from gymnasium.spaces import Box

# === Test Mode ===
test_mode = False  # True for debugging on a single stock
test_symbol = 'AAPL'

# === Custom Environment ===
class ContinuousTradingEnv(StocksEnv):
    def __init__(self, df, frame_bound, window_size):
        super().__init__(df=df, frame_bound=frame_bound, window_size=window_size)
        self.action_space = Box(low=-1.0, high=1.0, shape=(1,), dtype=np.float32)

    def _process_data(self):
        self.df = self.df.sort_values(by="Datetime")
        prices = self.df['Close'].values
        signal_features = self.df.drop(columns=['Symbol', 'Datetime', 'Close'], errors='ignore').select_dtypes(include=np.number).values
        return prices, signal_features

    def step(self, action):
        try:
            action = float(np.array(action).flatten()[0])
            if action < -0.3:
                discrete_action = 0  # SELL
            elif action > 0.3:
                discrete_action = 1  # BUY
            else:
                discrete_action = 2  # HOLD

            step_result = super().step(discrete_action)
            if len(step_result) == 5:
                obs, reward, terminated, truncated, info = step_result
            else:
                obs, reward, done, info = step_result
                terminated, truncated = done, False

            if self._current_tick > 0:
                price_diff = self.df['Close'].iloc[self._current_tick] - self.df['Close'].iloc[self._current_tick - 1]
                reward += price_diff * (1 if discrete_action == 1 else -1 if discrete_action == 0 else 0)
            reward = float(np.clip(reward, -1.0, 1.0))

            obs = np.array(obs, dtype=np.float32)
            info["terminal_observation"] = obs.copy()
            return obs, reward, terminated, truncated, info

        except Exception as e:
            obs = self.reset()[0] if isinstance(self.reset(), tuple) else self.reset()
            return obs, 0.0, True, True, {"terminal_observation": obs}

# === Google Drive Setup ===
drive.mount('/content/drive', force_remount=True)
SAVE_DIR = "/content/drive/MyDrive/Results_May_2025/results_sac_walkforward/sac_walkforward_models"
RESULTS_DIR = "/content/drive/MyDrive/Results_May_2025/results_sac_walkforward"
os.makedirs(SAVE_DIR, exist_ok=True)
os.makedirs(f"{RESULTS_DIR}/scalers", exist_ok=True)
os.makedirs(f"{RESULTS_DIR}/features", exist_ok=True)
os.makedirs(f"{RESULTS_DIR}/signals", exist_ok=True)
os.makedirs(f"{RESULTS_DIR}/plots", exist_ok=True)

# === Load Feature-Engineered Dataset ===
df = pd.read_csv("multi_stock_feature_engineered_dataset.csv")
df['Datetime'] = pd.to_datetime(df['Datetime'])
symbols = [test_symbol] if test_mode else df['Symbol'].unique()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
policy_kwargs = dict(net_arch=[64, 64])
results = []

# === Walkforward Dates ===
test_end_dt = datetime.today()
train_start_dt = test_end_dt - timedelta(days=729)
train_end_dt = train_start_dt + timedelta(days=365)
test_start_dt = train_end_dt
train_start = train_start_dt.strftime("%Y-%m-%d")
train_end = train_end_dt.strftime("%Y-%m-%d")
test_start = test_start_dt.strftime("%Y-%m-%d")
test_end = test_end_dt.strftime("%Y-%m-%d")
print(f"\n Walkforward Dates — Train: {train_start} to {train_end}, Test: {test_start} to {test_end}")

# === Main Loop ===
for symbol in symbols:
    print(f"\n Processing {symbol}")
    df_sym = df[df['Symbol'] == symbol].dropna().sort_values("Datetime").reset_index(drop=True)

    train_df = df_sym[(df_sym['Datetime'] >= train_start) & (df_sym['Datetime'] < train_end)].copy()
    test_df = df_sym[(df_sym['Datetime'] >= test_start) & (df_sym['Datetime'] <= test_end)].copy()

    if len(train_df) < 500 or len(test_df) < 200:
        print(f" Skipping {symbol} — not enough data.")
        continue

    # Drop text columns & non-numeric data
    train_df.drop(columns=['Session'], inplace=True, errors='ignore')
    test_df.drop(columns=['Session'], inplace=True, errors='ignore')

    features = [c for c in train_df.columns if c not in ['Symbol', 'Datetime', 'Close']]
    features = [c for c in features if np.issubdtype(train_df[c].dtype, np.number)]

    scaler = StandardScaler()
    train_df[features] = scaler.fit_transform(train_df[features])
    test_df[features] = scaler.transform(test_df[features])
    joblib.dump(scaler, f"{RESULTS_DIR}/scalers/{symbol}_scaler.pkl")
    with open(f"{RESULTS_DIR}/features/{symbol}_features.json", "w") as f:
        json.dump(features, f)

    # Train SAC
    env = DummyVecEnv([lambda: ContinuousTradingEnv(train_df, (50, len(train_df)), 10)])
    eval_env = DummyVecEnv([lambda: Monitor(ContinuousTradingEnv(test_df, (50, len(test_df)), 10))])
    noise = NormalActionNoise(mean=np.zeros(env.action_space.shape[-1]), sigma=0.2 * np.ones(env.action_space.shape[-1]))

    model = SAC("MlpPolicy", env, device=device, batch_size=256, buffer_size=50000,
                train_freq=1, gradient_steps=1, learning_rate=3e-4, gamma=0.99,
                tau=0.005, action_noise=noise, policy_kwargs=policy_kwargs, verbose=0)

    callback = EvalCallback(eval_env, best_model_save_path=f"./tmp_best_sac_{symbol}", eval_freq=1000,
                            callback_after_eval=StopTrainingOnNoModelImprovement(4, 6, verbose=1), verbose=0)

    model.learn(total_timesteps=min(25000, len(train_df) * 10), callback=callback)
    model.save(f"{SAVE_DIR}/sac_{symbol}")

    # Evaluation
    env = DummyVecEnv([lambda: ContinuousTradingEnv(test_df, (50, len(test_df)), 10)])
    obs = env.reset()
    capital, shares = 100000, 0
    portfolio, y_true, y_pred = [], [], []
    buy_hold = 100000 * (test_df['Close'].iloc[-1] / test_df['Close'].iloc[0])

    for i in range(len(test_df)):
        action, _ = model.predict(obs, deterministic=True)
        step_result = env.step(action)
        if len(step_result) == 5:
            obs, _, done, _, _ = step_result
        else:
            obs, _, done, _ = step_result
            done = [done]
        price = test_df['Close'].iloc[i]
        signal = action[0][0]
        y_pred.append(1 if signal > 0.3 else -1 if signal < -0.3 else 0)
        y_true.append(1 if i > 0 and price > test_df['Close'].iloc[i - 1] else -1 if i > 0 and price < test_df['Close'].iloc[i - 1] else 0)
        if signal > 0.3 and shares == 0:
            shares = capital // price
            capital -= shares * price
        elif signal < -0.3 and shares > 0:
            capital += shares * price
            shares = 0
        portfolio.append(capital + shares * price)
        if done[0]:
            break

    aligned_len = min(len(test_df), len(portfolio))
    test_df = test_df.iloc[:aligned_len].copy()
    test_df["Portfolio"] = portfolio[:aligned_len]
    test_df.to_csv(f"{RESULTS_DIR}/signals/{symbol}_sac_signals.csv", index=False)

    # Metrics
    y_true, y_pred = y_true[:aligned_len], y_pred[:aligned_len]
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='macro', zero_division=0)
    recall = recall_score(y_true, y_pred, average='macro', zero_division=0)
    f1 = f1_score(y_true, y_pred, average='macro', zero_division=0)
    final_value = portfolio[-1]
    returns = pd.Series(portfolio).pct_change().dropna()
    sharpe = (returns.mean() / returns.std()) * np.sqrt(252) if returns.std() > 0 else 0
    drawdown = ((pd.Series(portfolio).cummax() - pd.Series(portfolio)) / pd.Series(portfolio).cummax()).max() * 100
    ret = (final_value - 100000) / 100000

    # Plot
    plt.figure(figsize=(10, 4))
    plt.plot(portfolio, label='SAC Portfolio')
    plt.axhline(y=buy_hold, color='r', linestyle='--', label='Buy & Hold')
    plt.title(f"{symbol} Portfolio vs Buy & Hold")
    plt.legend()
    plt.grid()
    plt.tight_layout()
    plt.savefig(f"{RESULTS_DIR}/plots/{symbol}_portfolio.png")
    plt.close()

    results.append({
        "Ticker": symbol,
        "Model": "SAC",
        "Window": f"{train_start} to {test_end}",
        "Return": round(ret, 4),
        "sac_Portfolio": round(final_value, 2),
        "BuyHold": round(buy_hold, 2),
        "Sharpe": round(sharpe, 3),
        "Drawdown": round(drawdown, 2),
        "Accuracy": round(accuracy, 3),
        "Precision": round(precision, 3),
        "Recall": round(recall, 3),
        "F1_Score": round(f1, 3),
        "score": round(sharpe * 0.4 + final_value * 0.000003 - drawdown * 0.3, 4),
        "Winner": "SAC" if final_value > buy_hold else "Buy & Hold"
    })

    # Free up memory
    del model, env, eval_env
    gc.collect()
    torch.cuda.empty_cache()

# Save summary
results_df = pd.DataFrame(results)
results_df.to_csv(f"{RESULTS_DIR}/sac_performance_summary.csv", index=False)
print("\n All results saved.")


Mounted at /content/drive

 Walkforward Dates — Train: 2023-06-09 to 2024-06-08, Test: 2024-06-08 to 2025-06-07

 Processing AAPL

 Processing TSLA
Stopping training because there was no new best model in the last 5 evaluations

 Processing MSFT

 Processing GOOGL
Stopping training because there was no new best model in the last 5 evaluations

 Processing AMZN

 Processing NVDA
Stopping training because there was no new best model in the last 5 evaluations

 Processing META
Stopping training because there was no new best model in the last 5 evaluations

 Processing BRK-B

 Processing JPM

 Processing JNJ

 Processing XOM

 Processing V

 Processing PG

 Processing UNH

 Processing MA

 Processing HD

 Processing LLY

 Processing MRK

 Processing PEP

 Processing KO


KeyboardInterrupt: 

In [17]:
import os
import pandas as pd
import joblib
import json
from stable_baselines3 import SAC
from google.colab import drive
# === Google Drive Setup ===
drive.mount('/content/drive', force_remount=True)
# === Resume Script Config ===
SAVE_DIR = "/content/drive/MyDrive/Results_May_2025/results_sac_walkforward/sac_walkforward_models"
RESULTS_DIR = "/content/drive/MyDrive/Results_May_2025/results_sac_walkforward"
DATA_PATH = "multi_stock_feature_engineered_dataset.csv"

# === Load Previous Dataset ===
df = pd.read_csv(DATA_PATH)
df['Datetime'] = pd.to_datetime(df['Datetime'])
symbols = df['Symbol'].unique()

completed_symbols = [f.split("sac_")[1].split(".")[0] for f in os.listdir(SAVE_DIR) if f.endswith(".zip")]
remaining_symbols = [s for s in symbols if s not in completed_symbols]

print(f"\n Completed: {len(completed_symbols)} symbols")
print(f" Resuming with: {len(remaining_symbols)} symbols\n")

# === Reuse Config and Imports ===
from datetime import datetime, timedelta
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import torch
import numpy as np
import matplotlib.pyplot as plt
import gc
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnNoModelImprovement
from stable_baselines3.common.noise import NormalActionNoise
from gym_anytrading.envs import StocksEnv
from gymnasium.spaces import Box

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
policy_kwargs = dict(net_arch=[64, 64])
base_columns = ['Datetime', 'Open', 'High', 'Low', 'Close', 'Volume']
results = []

# === Define Custom Env (same as in main script) ===
class ContinuousTradingEnv(StocksEnv):
    def __init__(self, df, frame_bound, window_size):
        super().__init__(df=df, frame_bound=frame_bound, window_size=window_size)
        self.action_space = Box(low=-1.0, high=1.0, shape=(1,), dtype=np.float32)

    def _process_data(self):
        self.df = self.df.sort_values(by="Datetime")
        prices = self.df['Close'].values
        signal_features = self.df.drop(columns=['Symbol', 'Datetime', 'Close'], errors='ignore').values
        return prices, signal_features

    def step(self, action):
        try:
            action = float(np.array(action).flatten()[0])
            if action < -0.3:
                discrete_action = 0
            elif action > 0.3:
                discrete_action = 1
            else:
                discrete_action = 2

            step_result = super().step(discrete_action)
            if len(step_result) == 5:
                obs, reward, terminated, truncated, info = step_result
            else:
                obs, reward, done, info = step_result
                terminated, truncated = done, False

            if self._current_tick > 0:
                price_diff = self.df['Close'].iloc[self._current_tick] - self.df['Close'].iloc[self._current_tick - 1]
                reward += price_diff * (1 if discrete_action == 1 else -1 if discrete_action == 0 else 0)
            reward = float(np.clip(reward, -1.0, 1.0))

            obs = np.array(obs, dtype=np.float32)
            info["terminal_observation"] = obs.copy()
            return obs, reward, terminated, truncated, info

        except Exception as e:
            obs = self.reset()[0] if isinstance(self.reset(), tuple) else self.reset()
            return obs, 0.0, True, True, {"terminal_observation": obs}

# === Walkforward Dates ===
test_end_dt = datetime.today()
train_start_dt = test_end_dt - timedelta(days=729)
train_end_dt = train_start_dt + timedelta(days=365)
test_start_dt = train_end_dt
test_end = test_end_dt.strftime("%Y-%m-%d")
train_start = train_start_dt.strftime("%Y-%m-%d")
train_end = train_end_dt.strftime("%Y-%m-%d")
test_start = test_start_dt.strftime("%Y-%m-%d")

# === Resume Loop ===
for symbol in remaining_symbols:
    print(f"\n Resuming {symbol}")
    df_sym = df[df['Symbol'] == symbol].dropna().sort_values("Datetime").reset_index(drop=True)
    train_df = df_sym[(df_sym['Datetime'] >= train_start) & (df_sym['Datetime'] < train_end)].copy()
    test_df = df_sym[(df_sym['Datetime'] >= test_start) & (df_sym['Datetime'] <= test_end)].copy()

    if len(train_df) < 500 or len(test_df) < 200:
        print(f" Skipping {symbol} — insufficient data.")
        continue

    features = [c for c in train_df.columns if c not in ['Close', 'Symbol', 'Datetime', 'Session']]
    features = [c for c in features if np.issubdtype(train_df[c].dtype, np.number)]
    train_df.drop(columns=['Session'], inplace=True, errors='ignore')
    test_df.drop(columns=['Session'], inplace=True, errors='ignore')

    scaler = StandardScaler()
    train_df[features] = scaler.fit_transform(train_df[features])
    test_df[features] = scaler.transform(test_df[features])
    joblib.dump(scaler, f"{RESULTS_DIR}/scalers/{symbol}_scaler.pkl")
    with open(f"{RESULTS_DIR}/features/{symbol}_features.json", "w") as f:
        json.dump(features, f)

    env = DummyVecEnv([lambda: ContinuousTradingEnv(train_df, frame_bound=(50, len(train_df)), window_size=10)])
    eval_env = DummyVecEnv([lambda: Monitor(ContinuousTradingEnv(test_df, frame_bound=(50, len(test_df)), window_size=10))])
    action_noise = NormalActionNoise(mean=np.zeros(env.action_space.shape[-1]), sigma=0.2 * np.ones(env.action_space.shape[-1]))

    model = SAC("MlpPolicy", env, device=device, batch_size=256, buffer_size=50000,
                train_freq=1, gradient_steps=1, learning_rate=3e-4, gamma=0.99,
                tau=0.005, action_noise=action_noise, policy_kwargs=policy_kwargs, verbose=0)

    callback = EvalCallback(eval_env, best_model_save_path=f"./tmp_best_sac_{symbol}", eval_freq=1000,
                            callback_after_eval=StopTrainingOnNoModelImprovement(4, 6, verbose=1), verbose=0)

    model.learn(total_timesteps=min(25000, len(train_df) * 10), callback=callback)
    model.save(f"{SAVE_DIR}/sac_{symbol}")

    # === Evaluate ===
    obs = DummyVecEnv([lambda: ContinuousTradingEnv(test_df, frame_bound=(50, len(test_df)), window_size=10)]).reset()
    capital, shares = 100000, 0
    portfolio, y_true, y_pred = [], [], []
    buy_hold = 100000 * (test_df['Close'].iloc[-1] / test_df['Close'].iloc[0])

    for i in range(len(test_df)):
        action, _ = model.predict(obs, deterministic=True)
        step_result = env.step(action)
        if len(step_result) == 5:
            obs, _, done, _, _ = step_result
        else:
            obs, _, done, _ = step_result
            done = [done]

        price = test_df['Close'].iloc[i]
        signal = action[0][0]
        y_pred.append(1 if signal > 0.3 else -1 if signal < -0.3 else 0)
        y_true.append(1 if i > 0 and test_df['Close'].iloc[i] > test_df['Close'].iloc[i - 1] else -1 if i > 0 and test_df['Close'].iloc[i] < test_df['Close'].iloc[i - 1] else 0)
        if signal > 0.3 and shares == 0:
            shares = capital // price
            capital -= shares * price
        elif signal < -0.3 and shares > 0:
            capital += shares * price
            shares = 0
        portfolio.append(capital + shares * price)
        if done[0]:
            break

    aligned_len = min(len(test_df), len(portfolio))
    test_df = test_df.iloc[:aligned_len].copy()
    test_df["Portfolio"] = portfolio[:aligned_len]
    test_df.to_csv(f"{RESULTS_DIR}/signals/{symbol}_sac_signals.csv", index=False)

    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='macro', zero_division=0)
    recall = recall_score(y_true, y_pred, average='macro', zero_division=0)
    f1 = f1_score(y_true, y_pred, average='macro', zero_division=0)

    final_value = portfolio[-1]
    returns = pd.Series(portfolio).pct_change().dropna()
    sharpe = (returns.mean() / returns.std()) * np.sqrt(252) if returns.std() > 0 else 0
    drawdown = ((pd.Series(portfolio).cummax() - pd.Series(portfolio)) / pd.Series(portfolio).cummax()).max() * 100
    ret = (final_value - 100000) / 100000

    plt.figure(figsize=(10, 4))
    plt.plot(portfolio, label='SAC Portfolio')
    plt.axhline(y=buy_hold, color='r', linestyle='--', label='Buy & Hold')
    plt.title(f"{symbol} Portfolio vs Buy & Hold")
    plt.legend()
    plt.grid()
    plt.tight_layout()
    plt.savefig(f"{RESULTS_DIR}/plots/{symbol}_portfolio.png")
    plt.close()

    results.append({
        "Ticker": symbol,
        "Model": "SAC",
        "Window": f"{train_start} to {test_end}",
        "Return": round(ret, 4),
        "sac_Portfolio": round(final_value, 2),
        "BuyHold": round(buy_hold, 2),
        "Sharpe": round(sharpe, 3),
        "Drawdown": round(drawdown, 2),
        "Accuracy": round(accuracy, 3),
        "Precision": round(precision, 3),
        "Recall": round(recall, 3),
        "F1_Score": round(f1, 3),
        "score": round(sharpe * 0.4 + final_value * 0.000003 - drawdown * 0.3, 4),
        "Winner": "SAC" if final_value > buy_hold else "Buy & Hold"
    })

    gc.collect()
    torch.cuda.empty_cache()

# === Save Results ===
results_df = pd.DataFrame(results)
results_df.to_csv(f"{RESULTS_DIR}/sac_performance_resume_summary.csv", index=False)
print("\n Resume run complete and results saved.")


Mounted at /content/drive

 Completed: 39 symbols
 Resuming with: 14 symbols


 Resuming TXN
Stopping training because there was no new best model in the last 5 evaluations

 Resuming AMGN
Stopping training because there was no new best model in the last 5 evaluations

 Resuming UPS
Stopping training because there was no new best model in the last 5 evaluations

 Resuming LIN

 Resuming PM

 Resuming UNP

 Resuming BMY
Stopping training because there was no new best model in the last 5 evaluations

 Resuming LOW

 Resuming RTX

 Resuming CVX

 Resuming IBM
Stopping training because there was no new best model in the last 5 evaluations

 Resuming GE
Stopping training because there was no new best model in the last 5 evaluations

 Resuming SBUX
Stopping training because there was no new best model in the last 5 evaluations

 Resuming ORCL

 Resume run complete and results saved.


In [14]:
!fusermount -u /content/drive


fusermount: failed to unmount /content/drive: Invalid argument


In [16]:
!rm -rf /content/drive
